In [2]:
import azureml.core
from azureml.core import Workspace

ws = Workspace.from_config()

print(f"Working in Azure workspace {ws.name} with version {azureml.core.VERSION}")

Working in Azure workspace azml-sdk with version 1.30.0


### Method 3: Running an experiment interactively with MLflow

MLflow is an alternative to the native log functions. It is a platform agnostic framework for end2end ML work flow. 

In [ ]:
from azureml.core import Experiment
import pandas as pd
import mlflow

mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())

print(ws.get_mlflow_tracking_uri())

experiment = Experiment(workspace=ws, name="diabetes-local-mlflow")
mlflow.set_experiment(experiment.name)

with mlflow.start_run():
    print(f"Starting experiment: {experiment.name}")
    
    df = pd.read_csv("data/diabetes.csv")
    
    
    # Log some basic metrics
    row_count = df.shape[0]
    mlflow.log_metric("observations", row_count)  # Log a number
#     for col in df.columns:
#         mlflow.log_param("data columns", col)  # Log a list of strings
#     mlflow.log_metric("categorical columns", cat_cols)
#     mlflow.log_metric("numerical columns", num_cols)

#     # Log descriptive statistics
#     summary_stats = df[num_cols].describe().to_dict()
#     for col_name, stat_dict in summary_stats.items():
#         for stat_name, val in stat_dict.items():
#             run.log_row(col_name, stat=stat_name, value=val)  # Logging rows


#     # Log some image output
#     import matplotlib.pyplot as plt

#     diabetic_counts = df['Diabetic'].value_counts()

#     fig = plt.figure(figsize=(6,6))
#     ax = fig.gca()    
#     diabetic_counts.plot.bar(ax = ax) 
#     ax.set_title('Patients with Diabetes') 
#     ax.set_xlabel('Diagnosis') 
#     ax.set_ylabel('Patients')

#     run.log_image(name='label distribution', plot=fig)  # Log an image

#     # Log some file
#     os.makedirs("output", exist_ok=True)
#     df.sample(100).to_csv("output/sample.csv", index=False)